In [0]:
%matplotlib inline
# !pip install -U plotly
# import plotly.express as px
from urllib.request import urlopen

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import pandas
import time
import io
import json

#You will need to mount the drive to use the relevant datasets in the shared 'Data' folder
from google.colab import drive
drive.mount('/content/drive')

In [0]:
NYT_LINK = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
CENSUS_LINK = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv'
CENSUS_ENCODING = 'latin1'

nyt_cases = pd.read_csv(NYT_LINK) 
census_data = pd.read_csv(CENSUS_LINK, encoding=CENSUS_ENCODING)
pop_data = census_data[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','POPESTIMATE2019']]

pop_data["fips"] = pop_data['STATE']*1000 + pop_data['COUNTY'] 
case_pop = pd.merge(nyt_cases, pop_data[['POPESTIMATE2019', 'fips']], on = 'fips')
case_pop = case_pop.rename(columns = {'POPESTIMATE2019':'population'})

all_cases_time = case_pop.set_index(pd.to_datetime(case_pop.date))
del all_cases_time['date']

all_cases_time['population_adjusted_cases'] = all_cases_time['cases']/all_cases_time['population']
all_cases_time['fips'] = all_cases_time['fips'].astype(float).astype(int)
all_cases_time['fips'] = all_cases_time.fips.map("{:05}".format)

all_cases_time

In [0]:
# Helper function to get Y value given start_date and end_date
def get_difference_df(df, start_date, end_date):
  beginning_cases = all_cases_time[start_date]
  end_cases = all_cases_time[end_date]

  fips = list(set(beginning_cases.fips) & set(end_cases.fips)) #only want the cases recorded on both days
  beginning_cases = beginning_cases[beginning_cases.fips.isin(fips)]
  end_cases = end_cases[end_cases.fips.isin(fips)]
  end_cases["end_cases"] = end_cases.cases
  beginning_cases["beginning_cases"] = beginning_cases.cases

  case_difference = beginning_cases.reset_index()
  del case_difference['cases']
  del case_difference['date']

  case_difference['end_cases']= end_cases.reset_index().cases
  case_difference['end_deaths'] = end_cases.reset_index().deaths
  case_difference['case_diff'] = case_difference['end_cases']-case_difference['beginning_cases']
  case_difference['perc_increase'] = case_difference['case_diff']/case_difference['beginning_cases']*100
  case_difference['adj_increase'] = case_difference['case_diff']/case_difference['population']

  return case_difference

In [0]:
START_DATE = '2020-04-08'
END_DATE = '2020-04-15'
case_diff_1 = get_difference_df(all_cases_time, START_DATE, END_DATE)

START_DATE = '2020-04-15'
END_DATE = '2020-04-22'
case_diff_2 = get_difference_df(all_cases_time, START_DATE, END_DATE)

START_DATE = '2020-04-22'
END_DATE = '2020-04-29'
case_diff_3 = get_difference_df(all_cases_time, START_DATE, END_DATE)

In [0]:
case_diff = case_diff_1[['county', 'state', 'fips', 'population']]
case_diff.head()

In [0]:
case_diff['Y1_case_diff'] = float('nan')
case_diff['Y1_perc_increase'] = float('nan')
case_diff['Y1_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  if len(case_diff_1.loc[case_diff_1['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_case_diff'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_perc_increase'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_adj_increase'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

In [0]:
case_diff['Y2_case_diff'] = float('nan')
case_diff['Y2_perc_increase'] = float('nan')
case_diff['Y2_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  # print(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values)
  if len(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_case_diff'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_perc_increase'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_adj_increase'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

In [0]:
case_diff['Y3_case_diff'] = float('nan')
case_diff['Y3_perc_increase'] = float('nan')
case_diff['Y3_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  # print(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values)
  if len(case_diff_3.loc[case_diff_3['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_case_diff'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_perc_increase'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_adj_increase'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

In [0]:
case_diff.to_csv('case_diff_Y_v2.csv')

In [0]:
from google.colab import files
files.download('case_diff_Y_v2.csv')

UNACAST DATA AS T

In [0]:
county_unacast = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/county_unacast.csv')
county_unacast.head()

In [0]:
county_unacast.loc[county_unacast['county_fips']==53061, 'num_grade'].values.mean()
print(county_unacast['county_fips'])

In [0]:
unacast_T = case_diff[['county', 'state', 'fips', 'population']]
unacast_T.head()

In [0]:
unacast_T['fips']

In [0]:
unacast_T['unacast_score'] = 0
for idx, row in unacast_T.iterrows():
  if len(county_unacast.loc[county_unacast['county_fips']==int(row['fips']), 'num_grade'].values) > 0:
    unacast_T.loc[unacast_T['fips']==row['fips'], 'unacast_score'] = county_unacast.loc[county_unacast['county_fips']==int(row['fips']), 'num_grade'].values.mean()

unacast_T.head()

In [0]:
del unacast_T['population']
unacast_T.head()

In [0]:
unacast_T.to_csv('unacast_T.csv')
files.download('unacast_T.csv')

Covariates X

1. Demographics: population (total), male/female ratio, age distribution (18 classes; group 1 is 0-5, group 2 is 5-10, ...), race distribution (white, black, native american, asian, hispanic, ...)
2. Socioeconomics: Income, Education, Poverty
3. COVID-19: Current total number of confirmed cases/deaths, tests given, positive/negative test ratio, comorbidities

In [0]:
# Demographics: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/cc-est2018-alldata.pdf
#               cc-est2018-alldata.csv

demographic = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/cc-est2018-alldata.csv', encoding='latin1')
demographic.head()

In [0]:
# Things to note about processing this
# FIPS = 1000 * STATE + COUNTY
# YEAR: 11 is the most recent (July 2018)
# AGEGRP: 0 is total, 1~18 represent different age group
# Race: WA (White), BA (Black), IA (American + Alsaka Native), AA (Asian), NA (Native Hawaiian + Pacific Islander), H (Hispanic)

# What do we want:
# FIPS, TOTAL_MALE (%), TOTAL_FEMALE (%), AGE DISTRIBUTION (1~18, %), RACE DISTRIBUTION (W, B, I, A, N, H in %)

# 1) Filter by YEAR = 11
recent_demo = demographic.loc[demographic['YEAR']==11]
recent_demo

In [0]:
# 2) Create a FIPS column
recent_demo['fips'] = recent_demo['STATE'] * 1000 + recent_demo['COUNTY']
recent_demo['fips'] = recent_demo['fips'].astype(float).astype(int)
recent_demo['fips'] = recent_demo.fips.map("{:05}".format)
#all_cases_time['fips'] = all_cases_time['fips'].astype(float).astype(int)
#all_cases_time['fips'] = all_cases_time.fips.map("{:05}".format)
recent_demo.head()

In [0]:
# 3) Create a new df with fips, stname, ctyname
df = recent_demo[['fips', 'STNAME', 'CTYNAME']].drop_duplicates()
df.head()

In [0]:
temp = recent_demo.loc[recent_demo['fips']=='01001']
temp.loc[temp['AGEGRP']==0]['TOT_POP'].values[0]

In [0]:
# 4) Add relevant data
df['male'] = 0
df['female'] = 0

for i in range(1, 19, 1):
  age_grp = 'age_' + str(i)
  df[age_grp] = 0

df['white'] = 0
df['black'] = 0
df['native'] = 0
df['pacific'] = 0
df['asian'] = 0
df['hispanic'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  demo = recent_demo.loc[recent_demo['fips']==fips]
  total_pop = demo.loc[demo['AGEGRP']==0]['TOT_POP'].values[0]
  total_male = demo.loc[demo['AGEGRP']==0]['TOT_MALE'].values[0]
  total_female = demo.loc[demo['AGEGRP']==0]['TOT_FEMALE'].values[0]

  age = []
  for i in range(1, 19, 1):
    age.append(demo.loc[demo['AGEGRP']==i]['TOT_POP'].values[0])

  white = demo.loc[demo['AGEGRP']==0]['WA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['WA_FEMALE'].values[0]
  black = demo.loc[demo['AGEGRP']==0]['BA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['BA_FEMALE'].values[0]
  native = demo.loc[demo['AGEGRP']==0]['IA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['IA_FEMALE'].values[0]
  pacific = demo.loc[demo['AGEGRP']==0]['NA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['NA_FEMALE'].values[0]
  asian = demo.loc[demo['AGEGRP']==0]['AA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['AA_FEMALE'].values[0]
  hispanic = demo.loc[demo['AGEGRP']==0]['H_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['H_FEMALE'].values[0]

  df.loc[df['fips']==fips, 'male'] = float(total_male) / float(total_pop)
  df.loc[df['fips']==fips, 'female'] = float(total_female) / float(total_pop)

  for i in range(1, 19, 1):
    agegrp = 'age_' + str(i)
    df.loc[df['fips']==fips, agegrp] = float(age[i-1]) / float(total_pop)

  df.loc[df['fips']==fips, 'white'] = float(white) / float(total_pop)
  df.loc[df['fips']==fips, 'black'] = float(black) / float(total_pop)
  df.loc[df['fips']==fips, 'native'] = float(native) / float(total_pop)
  df.loc[df['fips']==fips, 'pacific'] = float(pacific) / float(total_pop)
  df.loc[df['fips']==fips, 'asian'] = float(asian) / float(total_pop)
  df.loc[df['fips']==fips, 'hispanic'] = float(hispanic) / float(total_pop)

  # df.head()


In [0]:
df['total_pop'] = 0
for idx, row in df.iterrows():
  fips = row['fips']
  demo = recent_demo.loc[recent_demo['fips']==fips]
  total_pop = demo.loc[demo['AGEGRP']==0]['TOT_POP'].values[0]
  df.loc[df['fips']==fips, 'total_pop'] = total_pop

df.head()

In [0]:
from google.colab import files
df.to_csv('demographics_X.csv')
files.download('demographics_X.csv')

In [0]:
df.head()

In [0]:
edu_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/Education.xls')
poverty_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/PovertyEstimates.xls')
income_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/Unemployment.xls')

In [0]:
edu_df = edu_df[4:]
edu_df = edu_df[['Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2018', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46']]
# edu_df['Unnamed: 46']
edu_df.head()

In [0]:
edu_df = edu_df.rename(columns = {'Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2018':'FIPS'})
edu_df = edu_df.rename(columns = {'Unnamed: 43':'LessHighSchool', 'Unnamed: 44':'OnlyHighSchool', 'Unnamed: 45':'SomeBachelor', 'Unnamed: 46':'Bachelor'})
edu_df.head()

In [0]:
poverty_df = poverty_df[4:]
poverty_df = poverty_df[['Poverty estimates for the U.S., States, and counties, 2018', 'Unnamed: 7']]

poverty_df.head()

In [0]:
poverty_df = poverty_df.rename(columns={'Poverty estimates for the U.S., States, and counties, 2018':'FIPS', 'Unnamed: 7':'Poverty'})
poverty_df.head()

In [0]:
income_df = income_df[4:]
income_df = income_df[['Unemployment and median household income for the U.S., States, and counties, 2007-18', 'Unnamed: 53', 'Unnamed: 54']]

income_df.head()

In [0]:
income_df = income_df.rename(columns={'Unemployment and median household income for the U.S., States, and counties, 2007-18':'FIPS', \
                                      'Unnamed: 53':'UnemploymentRate', 'Unnamed: 54':'MedianIncome'})
income_df.head()

In [0]:
df.head()

In [0]:
df['Education_LessHighSchool'] = 0
df['Education_OnlyHighSchool'] = 0
df['Education_SomeBachelor'] = 0
df['Education_Bachelor'] = 0

df['Poverty'] = 0
df['Unemployment'] = 0
df['MedianIncome'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  
  edu = edu_df.loc[edu_df['FIPS']==fips]
  if len(edu) > 0:
    df.loc[df['fips']==fips, 'Education_LessHighSchool'] = float(edu['LessHighSchool'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_OnlyHighSchool'] = float(edu['OnlyHighSchool'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_SomeBachelor'] = float(edu['SomeBachelor'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_Bachelor'] = float(edu['Bachelor'].values[0]) / 100.
  
  if len(poverty_df.loc[poverty_df['FIPS']==fips]) > 0:
    df.loc[df['fips']==fips, 'Poverty'] = float(poverty_df.loc[poverty_df['FIPS']==fips]['Poverty'].values[0]) / float(row['total_pop'])

  if len(income_df.loc[income_df['FIPS']==fips]) > 0:
    df.loc[df['fips']==fips, 'Unemployment'] = float(income_df.loc[income_df['FIPS']==fips]['UnemploymentRate'].values[0]) / 100. 
    df.loc[df['fips']==fips, 'MedianIncome'] = income_df.loc[income_df['FIPS']==fips]['MedianIncome'].values[0]

df.head()



In [0]:
df.to_csv('demographics_socioecon_X.csv')
files.download('demographics_socioecon_X.csv')

Combining demographics + socioecon + current COVID status + politics

In [0]:
df = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/demographics_socioecon_X.csv')
del df['Unnamed: 0']
df.head()

In [0]:
pop_density_per_county = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/associated-data/master/us_census_data/us_census_2018_population_estimates_counties.csv')
pop_density_per_county.head()

In [0]:
state_tests = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv')
state_tests.head()

In [0]:
election_data = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv')
election_data.head()

In [0]:
election_data.loc[election_data['combined_fips']//1000 == 2]

In [0]:
df['pop_density'] = 0
df['total_test'] = 0
df['pos_test_rate'] = 0
df['dem_support'] = 0
df['gop_support'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  state_fips = int(fips / 1000)
  state_pop = df.loc[df['fips']//1000==state_fips]['total_pop'].values.sum()

  if len( pop_density_per_county.loc[pop_density_per_county['geo_id']==fips]) > 0:
    df.loc[df['fips']==fips, 'pop_density'] = pop_density_per_county.loc[pop_density_per_county['geo_id']==fips]['pop_density'].values[0]

  if len(state_tests.loc[state_tests['fips']==state_fips]) > 0:
    pos_test = state_tests.loc[state_tests['fips']==state_fips]['positive'].values[0]
    neg_test = state_tests.loc[state_tests['fips']==state_fips]['negative'].values[0]
    df.loc[df['fips']==fips, 'total_test'] = (float(pos_test) + float(neg_test)) / float(state_pop)
    df.loc[df['fips']==fips, 'pos_test_rate'] = float(pos_test) / (float(pos_test) + float(neg_test))
  # print(fips)
  if len(election_data.loc[election_data['combined_fips']==fips]) > 0:
    df.loc[df['fips']==fips, 'dem_support'] = election_data.loc[election_data['combined_fips']==fips]['per_dem'].values[0]
    df.loc[df['fips']==fips, 'gop_support'] = election_data.loc[election_data['combined_fips']==fips]['per_gop'].values[0]

df.head()

In [0]:
# Now, add covid related data
# 1) total case until March 28 (a week before Apr 4)
# 2) increase during March 23-28

START_DATE = '2020-03-25'
END_DATE = '2020-04-01'
case_diff_X = get_difference_df(all_cases_time, START_DATE, END_DATE)
case_diff_X.head()

In [0]:
df['fips'] = df['fips'].astype(float).astype(int)
df['fips'] = df.fips.map("{:05}".format)

In [0]:
data_from_end = all_cases_time['2020-04-01']

In [0]:
df['cases'] = 0
df['deaths'] = 0
df['perc_increase'] = 0
df['adj_increase'] = 0

for idx, row in df.iterrows():
  if len(data_from_end.loc[data_from_end['fips']==row['fips']]) > 0:
    df.loc[df['fips']==row['fips'], 'cases'] = data_from_end.loc[data_from_end['fips']==row['fips']]['cases'].values[0]
    df.loc[df['fips']==row['fips'], 'deaths'] = data_from_end.loc[data_from_end['fips']==row['fips']]['deaths'].values[0]
  if len(case_diff_X.loc[case_diff_X['fips']==row['fips']]) > 0:
    df.loc[df['fips']==row['fips'], 'perc_increase'] = case_diff_X.loc[case_diff_X['fips']==row['fips']]['perc_increase'].values[0]
    df.loc[df['fips']==row['fips'], 'adj_increase'] = case_diff_X.loc[case_diff_X['fips']==row['fips']]['adj_increase'].values[0]

In [0]:
df.head(30)

In [0]:
from google.colab import files
df.to_csv('all_X_ver2.csv')
files.download('all_X_ver2.csv')